In [1]:
#Customers who left are 1, current customers are 0
#Males are 0, females are 1
#might play around with using -1/1 instead

#LOOK UP KURTOSIS, THE GUY DID IT ON MONTHLY TENURE, LOOKED COOL, IDK


#NOT SURE IF I SHOULD ONEHOTENCODE OR JUST TOKENIZE AS 0-4 FOR CREDIT CARD TYPE
    #I went with tkenizing 0-4 for now


#Open-to-Buy: The difference between the credit limit assigned to a cardholder account and the present balance on the account.
    #DO RATIO BETWEEN THIS AND CREDIT LIMIT




In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


In [3]:

starter_df = pd.read_csv(Path("c://users/ajcth/documents/github/bank_churn_project/Resources/BankChurners.csv"))

starter_df = starter_df[starter_df != 'Unknown'].dropna()
starter_df.sample(5)


,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
7218,711716133,Attrited Customer,42,F,3,High School,Married,$40K - $60K,Blue,34,...,1585.0,1381,204.0,0.706,2360,40,0.379,0.871,0.990950,0.00905
9352,792139308,Existing Customer,48,F,3,Graduate,Single,$40K - $60K,Blue,42,...,9215.0,639,8576.0,0.646,13345,114,0.754,0.069,0.000199,0.99980
166,714264408,Existing Customer,47,F,3,Uneducated,Married,Less than $40K,Blue,36,...,1611.0,746,865.0,0.993,1413,29,0.706,0.463,0.000022,0.99998
2704,719455908,Existing Customer,48,M,3,Graduate,Married,$80K - $120K,Blue,38,...,31501.0,0,31501.0,0.698,3742,75,0.786,0.000,0.000314,0.99969
3151,787578183,Existing Customer,36,M,3,High School,Single,$80K - $120K,Blue,28,...,1438.3,0,1438.3,1.041,3204,68,0.838,0.000,0.000181,0.99982


In [4]:
starter_df['Education_Level'].value_counts()

Graduate         2591
High School      1653
Uneducated       1204
College           844
Post-Graduate     431
Doctorate         358
Name: Education_Level, dtype: int64

In [5]:
#Shorten column names so they display better
replacement_column_names = {'Attrition_Flag': 'Status',
        'Customer_Age': 'Age',
        'Education_Level': 'Education',
        'Dependent_count': 'Dependents',
        'Income_Category': 'Income',
        'Card_Category': 'Card Type',
        'Months_on_book': 'Tenure(month)'
        }
starter_df.rename(columns=replacement_column_names, inplace=True)

In [6]:
#Drop unknowns
starter_df = starter_df[starter_df.Marital_Status != 'Unknown']
starter_df = starter_df[starter_df.Education != 'Unknown']
starter_df = starter_df[starter_df.Income != 'Unknown']

#NOT USING RIGHT NOWReplace M/F with -1/1
#starter_df = starter_df.replace({'Gender' : {'M': '-1', 'F': '1'}})

#Replace M/F with 0,1
starter_df = starter_df.replace({'Gender' : {'M': '0', 'F': '1'}})

#Replace customer's status to 0 if they left and 1 if they stayed
starter_df = starter_df.replace({'Status' : {'Attrited Customer': 1, 'Existing Customer': 0}})





In [7]:
#use feature creation to make a ratio between dependents and income sources
#consider someone divorced as Single. only 10% of divorced people receive alimony payments
        #either a 1 (single/divorced) or a 2 (married)
        #divide that by the amount of dependents plus one, which represents how many people you take care of
                #i.e. a single person with no dependents only takes care of themself, so they are a 1. 
starter_df = starter_df.replace({'Marital_Status' : {'Divorced': 1, 'Single': 1, 'Married' : 2}})
starter_df['MaritalDependentRatio'] = ((starter_df['Marital_Status'] / (starter_df['Dependents'] + 1)) / 2).round(2)
starter_df.drop(columns=['Marital_Status', 'Dependents'], inplace=True)

#Replace card categories from Blue/Silver/Gold/Plat to 0/1/2/3/4
starter_df = starter_df.replace({'Card Type' : {'Blue': 0, 'Silver': 1, 'Gold' : 3, 'Platinum' : 4}})


#Drop irrelevant column
starter_df = starter_df.drop(columns='CLIENTNUM')

In [8]:
#use feature creation to make a ratio between avg open to buy and credit limit

starter_df['CreditUsage'] = (starter_df['Avg_Open_To_Buy'] / starter_df['Credit_Limit']).round(2)
starter_df.drop(columns=['Avg_Open_To_Buy', 'Credit_Limit'], inplace=True)

In [9]:
#use feature creation to make a ratio between transaction amount and trans count
starter_df['AvgTransValue'] = (starter_df['Total_Trans_Amt'] / starter_df['Total_Trans_Ct']).round(2)
starter_df.drop(columns=['Total_Trans_Amt', 'Total_Trans_Ct'], inplace=True)

In [10]:
#There education categories are vague/ambiguous 
#In 2021, the highest level of education of the population age 25 and older in the United States was distributed as follows: 
    #8.9% had less than a high school diploma or equivalent.
        #our dataset shows 17% uneducated
    #27.9% had high school graduate as their highest level of school completed. 
        #our dataset shows 23.3%
    #14.9% had completed some college but not a degree.
        #ours only has an ambiguous 'graduate' as 36.6%. could be this, could mean post-grad, term is used interchangeably.
    #10.5% had an associate degree as their highest level of school completed.
        #our dataset has no distinction between associate/undergrad. just says 'college'. ours shows 11.9% for college
    #23.5% had a bachelor’s degree as their highest degree.
        #ours shows 11.9% for college
    #14.4% had completed an advanced degree such as a master’s degree, professional degree or doctoral degree. 
        #ours shows 6.1% post-graduate, 5.1% doctorate, and the ambiguous 36.6% 'graduate'ArithmeticError


counts = starter_df['Education'].value_counts()
percs = starter_df['Education'].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
pd.concat([counts,percs], axis=1, keys=['count', 'percentage'])

,count,percentage
Graduate,2591,36.6%
High School,1653,23.3%
Uneducated,1204,17.0%
College,844,11.9%
Post-Graduate,431,6.1%
Doctorate,358,5.1%


In [11]:
#It's hard to tell what they mean by 'graduate'. Unfortunately, its the bulk of our data, so we should not drop it.
    #The existence of a 'college' column suggests that graduate could mean beyond college. 
        #Alternatively, it could mean you attended college but did not graduate, and 'graduate' represents standard undergrads.
#The fact that this such a high amount of the data (36.6%), its likely a mix of multiple categories not included, i.e. trade schools, associate degrees, dropouts, specializations
#We should try to create a weighting system that roughly ranks them based on years at school (?)
    #Essentially:
        #if you are uneducated or just high school, you are a 0.
        #if you are college or graduate, you are a 1
        #if you are post-grad or doctorate, you are a 2

#Change education: 0 if high school or less, 1 if college or graduate, 2 if post-grad or doctorate
starter_df = starter_df.replace({'Education' : {'Uneducated': 0, 'High School': 0, 'College' : 1, 'Graduate' : 1, 'Post-Graduate' : 2, 'Doctorate' : 2}})



In [12]:
starter_df = starter_df.dropna()

In [13]:
#use OneHotEncoder for the income categories
enc = OneHotEncoder(sparse=False)
categorical_variables = ['Income']
encoded_data = enc.fit_transform(starter_df[categorical_variables])

encoded_df = pd.DataFrame(
    encoded_data,
    columns = enc.get_feature_names(categorical_variables)
)

encoded_df = pd.concat(
    [
        starter_df,
        encoded_df
    ],
    axis=1
)
encoded_df.drop(columns=['Income'], inplace=True)
encoded_df = encoded_df.dropna()


c:\python_files\Anaconda3\envs\dev\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [14]:
encoded_df = encoded_df.dropna()
encoded_df.sample(15)

,Status,Age,Gender,Education,Card Type,Tenure(month),Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Total_Revolving_Bal,...,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2,MaritalDependentRatio,CreditUsage,AvgTransValue,Income_$120K +,Income_$40K - $60K,Income_$60K - $80K,Income_$80K - $120K,Income_Less than $40K
5353,0.0,54.0,1,0.0,0.0,36.0,5.0,4.0,2.0,1355.0,...,0.000232,0.999770,0.12,0.70,57.30,0.0,0.0,0.0,0.0,1.0
3067,0.0,48.0,1,0.0,0.0,38.0,6.0,1.0,3.0,1444.0,...,0.000099,0.999900,0.25,0.59,49.42,0.0,0.0,1.0,0.0,0.0
5989,0.0,37.0,0,0.0,0.0,27.0,6.0,3.0,2.0,759.0,...,0.000194,0.999810,0.20,0.47,51.70,0.0,0.0,1.0,0.0,0.0
2043,1.0,37.0,0,2.0,0.0,22.0,3.0,3.0,4.0,0.0,...,0.998440,0.001557,1.00,1.00,47.22,0.0,0.0,1.0,0.0,0.0
4424,1.0,52.0,1,2.0,0.0,43.0,3.0,2.0,4.0,1605.0,...,0.997890,0.002111,0.33,0.12,65.08,0.0,0.0,1.0,0.0,0.0
3508,0.0,53.0,0,1.0,0.0,45.0,5.0,3.0,2.0,888.0,...,0.000183,0.999820,0.10,0.94,58.61,0.0,1.0,0.0,0.0,0.0
4220,0.0,50.0,0,0.0,0.0,37.0,4.0,1.0,4.0,1792.0,...,0.000144,0.999860,0.25,0.82,68.90,0.0,0.0,0.0,0.0,1.0
4713,0.0,59.0,1,1.0,0.0,48.0,3.0,3.0,4.0,0.0,...,0.000443,0.999560,1.00,1.00,56.34,0.0,1.0,0.0,0.0,0.0
301,0.0,45.0,0,0.0,0.0,36.0,5.0,3.0,2.0,1343.0,...,0.000185,0.999820,0.33,0.40,44.60,0.0,1.0,0.0,0.0,0.0
6103,1.0,41.0,1,1.0,0.0,15.0,5.0,3.0,4.0,2517.0,...,0.998080,0.001916,0.25,0.42,48.09,0.0,0.0,1.0,0.0,0.0


In [15]:
scaler = StandardScaler()
encoded_df[['Tenure(month)', 'Total_Revolving_Bal']] = scaler.fit_transform(encoded_df[['Tenure(month)', 'Total_Revolving_Bal']])

In [17]:
#Remove columns that intimidate me

encoded_df.drop(columns=['Total_Relationship_Count', #¿que?
'Contacts_Count_12_mon', #Not a clue
'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2', #Some kind of wizardry, incantations
'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1', #I didnt make this and idk what it do or how it do it
'Total_Amt_Chng_Q4_Q1', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio', #Same as it ever was
],
inplace=True
)
encoded_df.sample(10)

In [19]:
y = encoded_df['Status']
X = encoded_df.drop(columns=['Status'])

In [20]:
#Use SMOTE to add synthetic data and balance our target feature value count
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

In [21]:
#Check the value counts
y.value_counts()

0.0    4311
1.0    4311
Name: Status, dtype: int64

In [22]:
#Split features and target in to training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [23]:
#Choose a model

#clf = LogisticRegression()
clf = RandomForestClassifier()

In [24]:
#Fit the model on the training data
clf.fit(X_train, y_train)

RandomForestClassifier()

In [25]:
#Make predictions on the test data
test_predictions = clf.predict(X_test)

In [26]:
# Evaluate the model's performance
accuracy = accuracy_score(test_predictions, y_test)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 91.70%


In [27]:
#Create a dataframe showing predictions vs test
comparison = pd.DataFrame({"Testing Data Predictions": test_predictions,
    "Testing Data Actual Targets": y_test})
comparison = comparison.sort_index(ascending=True)
comparison.tail(5)

,Testing Data Predictions,Testing Data Actual Targets
8602,1.0,1.0
8605,1.0,1.0
8607,1.0,1.0
8615,1.0,1.0
8616,1.0,1.0


In [28]:
#Generate a test matrix
test_matrix = confusion_matrix(y_test, test_predictions)
print(test_matrix)

[[1022   72]
 [ 107  955]]


In [29]:
# Create a testing classifiction report
testing_report = classification_report(y_test, test_predictions)

# Print the testing classification report
print(testing_report)

              precision    recall  f1-score   support

         0.0       0.91      0.93      0.92      1094
         1.0       0.93      0.90      0.91      1062

    accuracy                           0.92      2156
   macro avg       0.92      0.92      0.92      2156
weighted avg       0.92      0.92      0.92      2156

